Name: [Sevgi Öztürk]

I hereby declare that I observed the honour code of the university when preparing the homework.

## Pr?gr?mm?ng?H?m?w?rk 3

In this exercise we model a string of text using a Markov(1) model. For simplicity we only consider letters 'a-z'. Capital letters 'A-Z' are mapped to the corresponding ones. All remaining letters, symbols, numbers, including spaces, are denoted by '.'.


We have a probability table $T$ where $T_{i,j} = p(x_t = j | x_{t-1} = i)$  transition model of letters in English text for $t=1,2 \dots N$. Assume that the initial letter in a string is always a space denoted as $x_0 = \text{'.'}$. Such a model where the probability table is always the same is sometimes called a stationary model.

1. For a given $N$, write a program to sample random strings with letters $x_1, x_2, \dots, x_N$ from $p(x_{1:N}|x_0)$
1. Now suppose you are given strings with missing letters, where each missing letter is denoted by a question mark (or underscore, as below). Implement a method, that samples missing letters conditioned on observed ones, i.e., samples from $p(x_{-\alpha}|x_{\alpha})$ where $\alpha$ denotes indices of observed letters. For example, if the input is 't??.', we have $N=4$ and
$x_1 = \text{'t'}$ and $x_4 = \text{'.'}$, $\alpha=\{1,4\}$ and $-\alpha=\{2,3\}$. Your program may possibly generate the strings 'the.', 'twi.', 'tee.', etc. Hint: make sure to make use all data given and sample from the correct distribution. Implement the method and print the results for the test strings below. 
1. Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.
1. Discuss how you can improve the model to get better estimations.

## Programming Solution

$1$. For a given $N$, write a program to sample random strings with letters $x_1, x_2, \dots, x_N$ from $p(x_{1:N}|x_0)$

Since the only condition is to start with $x_0$, bunch of chars are generated directly from transition table $p$

In [7]:
import csv
import numpy as np
import functools as fnt
from IPython.display import display, Latex

def dotProdWCount(i, j):
    count = abs(i-j)-1
    if count==0:
        return arrT
    elif count>0:
        newArrT = arrT
        for k in range(0,count):
            newArrT = np.dot(newArrT,arrT)
        return newArrT
    elif count<0: # only if string finished with an unknown
        return 1
    
alphRange = 27
alphabet = [chr(indx+ord('a')) for indx in range(alphRange-1)]
alphabet.append('.')
letter2idx = {c:ind for ind,c in enumerate(alphabet)}

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)

arrT = np.array([[float(y) for y in x] for x in T])

ind = 26 # take . index as prior knowledge for x0
lenStr = 10 # generate string of that length
times = 20  # repeat the operation that times
for i in range(1,times):
    newToken = '.'
    for j in range(1,lenStr):
        matrix=dotProdWCount(0,j)
        row = matrix[ind,:]
        p = [row]            
        ch = np.random.choice(alphabet,1, p)
        newToken = newToken + ch[0]
    print(newToken)

.wnnd.qndm
.pjxyezhrt
.fmorkmrzc
.qrbttvrqi
.qnoklamgg
.tyvairubu
.tevpmfhaj
.govvigohr
.hoirasrtg
.yccizlkvp
.ffcoieuoh
.ibvbzgmze
.ebrsguml.
.knhwexxyz
.jlwnxrfsc
.lotrxernh
.hpevazc.e
.kibwqojht
.yngdyqtss


$2$. Now suppose you are given strings with missing letters, where each missing letter is denoted by a question mark (or underscore, as below). Implement a method, that samples missing letters conditioned on observed ones, i.e., samples from $p(x_{-\alpha}|x_{\alpha})$ where $\alpha$ denotes indices of observed letters. For example, if the input is 't??.', we have $N=4$ and
$x_1 = \text{'t'}$ and $x_4 = \text{'.'}$, $\alpha=\{1,4\}$ and $-\alpha=\{2,3\}$. Your program may possibly generate the strings 'the.', 'twi.', 'tee.', etc. Hint: make sure to make use all data given and sample from the correct distribution. Implement the method and print the results for the test strings below.

Since Markov(1) will be used as stated above, known characters will be regarded as prior knowledge to guess the new distribution of unknown characters. Dot character added to the beginning of example strings just in case they would start with unknown character '_'.
For the first string example : $th__br__n.f_x.$
$$p(x_3|x_2='h',x_5='b')= \frac{p(x_3|x_2='h')\sum_{x_4} {p(x_5='b'|x_4)p(x_4|x_3)}}{\sum_{x_3,x_4} {p(x_5='b'|x_4)p(x_4|x_3)p(x_3|x_2='h')}} $$

Proceeding with the same logic : $a____b$
$$p(x_i|x_{startInd}='a',x_{endInd}='b')= \frac{\sum{x_{unknownsLessThani}} p(x_i|x_{i-1},\dots,p(x_{startInd+1}|x_{startInd}='a')\sum_{x_{unknownsGreaterThani}} p(x_{i+1}|x_{i},\dots,p(x_{endInd}='b'|x_{endInd-1})}{\sum_{x_{unknowns}} {p(x_{endInd}='b'|x_{endInd-1}) \dots p(x_{startInd+1}|x_{startInd}='a')}} $$

In [8]:
import csv
import numpy as np
import functools as fnt
from IPython.display import display, Latex

def dotProdWCount(i, j):
    count = abs(i-j)-1
    if count==0:
        return arrT
    elif count>0:
        newArrT = arrT
        for k in range(0,count):
            newArrT = np.dot(newArrT,arrT)
        return newArrT
    elif count<0: # only if string finished with an unknown
        return 1
    
alphRange = 27
alphabet = [chr(indx+ord('a')) for indx in range(alphRange-1)]
alphabet.append('.')
letter2idx = {c:ind for ind,c in enumerate(alphabet)}

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)

test_strings = ['th__br__n.f_x.','_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

arrT = np.array([[float(y) for y in x] for x in T])

for i in range(len(test_strings)):
    token = list('.' + test_strings[i])
    newToken = ''
    for j in range(len(token)):
        if token[j] != '_':        
            startInd = j # first known character to be based on    
            newToken = newToken + token[j]
        elif token[j] == '_':            
            for k in range(j,len(token)):
                if token[k]!='_':
                    endInd = k
                    break
                else:
                    endInd = j
            #print('j:',j, 'start:',startInd, ' endInd:',endInd)
            leftMatrix = dotProdWCount(startInd,j)
            rightMatrix = dotProdWCount(j,endInd)
            if endInd!=j:
                normMatrix = dotProdWCount(startInd,endInd)                   
                startIndMatrix = letter2idx[token[startInd]]
                endIndMatrix = letter2idx[token[endInd]]
                row = leftMatrix[startIndMatrix,:]
                column = rightMatrix[:,endIndMatrix]
                p = [row*column/normMatrix[startIndMatrix,endIndMatrix]]
            else: # finishes with an unknown char
                startIndMatrix = letter2idx[token[startInd]]
                row = leftMatrix[startIndMatrix,:]
                p = [row]            
            ch = np.random.choice(alphabet,1, p)
            newToken = newToken + ch[0]
            
    print(''.join(token), '==>', newToken)

.th__br__n.f_x. ==> .thumbrwrn.fjx.
._u_st__n_.to_be._nsw_r__ ==> .vubstzjnv.toxbe.rnswyrpl
.i__at_._a_h_n_._e_r_i_g ==> .icxato.yaihsnr.oeyrcing
.q___t.___z._____t.__.___.__. ==> .qvnxt.aauz.xlbdft.iy.csy.bw.


$3.$ Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.

In [10]:
# -*- coding: utf-8 -*-
"""
Created on Sun Mar 12 12:25:09 2017

@author: sevgi
"""

# -*- coding: utf-8 -*-
"""
Created on Sat Mar 11 18:55:18 2017

@author: sevgi
"""

import csv
import numpy as np
import functools as fnt
from IPython.display import display, Latex

def dotProdWCount(i, j):
    count = abs(i-j)-1
    if count==0:
        return arrT
    elif count>0:
        newArrT = arrT
        for k in range(0,count):
            newArrT = np.dot(newArrT,arrT)
        return newArrT
    elif count<0: # only if string finished with an unknown
        return 1
    
alphRange = 27
alphabet = [chr(indx+ord('a')) for indx in range(alphRange-1)]
alphabet.append('.')
letter2idx = {c:ind for ind,c in enumerate(alphabet)}

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)

test_strings = ['th__br__n.f_x.','_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

arrT = np.array([[float(y) for y in x] for x in T])

for i in range(len(test_strings)):
    token = list('.' + test_strings[i])
    newToken = ''
    logProb = 0
    for j in range(len(token)):
        if token[j] != '_':        
            startInd = j # first known character to be based on    
            newToken = newToken + token[j]
        elif token[j] == '_':            
            for k in range(j,len(token)):
                if token[k]!='_':
                    endInd = k
                    break
                else:
                    endInd = j
            #print('j:',j, 'start:',startInd, ' endInd:',endInd)
            leftMatrix = dotProdWCount(startInd,j)
            rightMatrix = dotProdWCount(j,endInd)
            if endInd!=j:
                normMatrix = dotProdWCount(startInd,endInd)                   
                startIndMatrix = letter2idx[token[startInd]]
                endIndMatrix = letter2idx[token[endInd]]
                row = leftMatrix[startIndMatrix,:]
                column = rightMatrix[:,endIndMatrix]
                p = [row*column/normMatrix[startIndMatrix,endIndMatrix]]
            else: # finishes with an unknown char
                startIndMatrix = letter2idx[token[startInd]]
                row = leftMatrix[startIndMatrix,:]
                p = [row]                        
            mx = max(p[0])
            logProb = logProb + np.log(mx)
            maxInd = np.where(p[0]==mx)[0]
            ch = alphabet[maxInd[0]]
            newToken = newToken + ch[0]
            
    print(''.join(token), '==>', newToken, ' log:',logProb)

.th__br__n.f_x. ==> .the.br.an.fex.  log: -3.58382057262
._u_st__n_.to_be._nsw_r__ ==> .oursthand.to.be.answere.  log: -11.4410966006
.i__at_._a_h_n_._e_r_i_g ==> .in.ath.wathend.he.r.ing  log: -11.818977991
.q___t.___z._____t.__.___.__. ==> .qur.t.terz.t....t.ae.t.e.ae.  log: -27.5276801803


$4$.Discuss how you can improve the model to get better estimations.

Instead of using Markov(1), it can be developed by using Markov(N), which contributes more to the following vertices.